In [ ]:
!pip install transformers==4.40.1 datasets==2.19.0 accelerate==0.29.3 peft==0.10.0 bitsandbytes==0.43.1 tiktoken==0.6.0 autotrain-advanced==0.7.77

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 109.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 25.8 MB/s eta 0:00:00
 

In [ ]:
import os
import torch
import transformers
from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    TextStreamer,
    pipeline
)

In [ ]:
def get_summary(model_name, prompt):
    print("============= MODEL NAME =============")
    print(model_name)
    print("======================================")

    # NF4 양자화를 위한 설정
    nf4_config = BitsAndBytesConfig(
        load_in_4bit=True, # 모델을 4비트 정밀도로 로드
        bnb_4bit_quant_type="nf4", # 4비트 NormalFloat 양자화: 양자화된 파라미터의 분포 범위를 정규분포 내로 억제하여 정밀도 저하 방지
        bnb_4bit_use_double_quant=True, # 이중 양자화: 양자화를 적용하는 정수에 대해서도 양자화 적용
        bnb_4bit_compute_dtype=torch.bfloat16 # 연산 속도를 높이기 위해 사용 (default: torch.float32)
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=nf4_config,
        device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    print("============= LOADED MODEL =============")
    print(model)
    print("========================================")

    # 텍스트 생성을 위한 파이프라인 설정
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1000) # max_new_tokens: 생성할 최대 토큰 수
    outputs = pipe(
        prompt,
        do_sample=True, # 샘플링 전략 사용. 확률 분포를 기반으로 다음 토큰을 선택
        temperature=0.2, # 샘플링의 다양성을 조절하는 파라미터. 값이 높을수록 랜덤성 증가
        top_k=50, # 다음 토큰을 선택할 때 상위 k개의 후보 토큰 중에서 선택. 여기에서는 상위 50개의 후보 토큰 중에서 샘플링
        top_p=0.95, # 누적 확률이 p가 될 때까지 후보 토큰을 포함
        repetition_penalty=1.2, # 반복 패널티를 적용하여 같은 단어나 구절이 반복되는 것 방지
        add_special_tokens=True # 모델이 입력 프롬프트의 시작과 끝을 명확히 인식할 수 있도록 특별 토큰 추가
    )
    print("============= MODEL OUTPUTS =============")
    print(outputs[0]["generated_text"][len(prompt):]) # 입력 프롬프트 이후에 생성된 텍스트만 출력
    print("=========================================")

In [ ]:
prompt = """아래는 작업을 설명하는 지시사항과 회의록 전문이 기록된 회의록이 짝으로 구성됩니다. 이에 대한 적절한 요약을 작성하세요.
### 지시사항
당신은 회의록을 요약해주는 유능한 회의 어시스턴트 AI입니다.
아래 주어진 회의록에서 중요한 내용을 반드시 포함하여 적절히 요악하세요.


### 회의록:
**일시:** 2023년 10월 15일 오후 2시
**장소:** 본사 회의실
**참석자:**
- **김대표** (김)
- **이팀장** (이)
- **박대리** (박)
- **최사원** (최)

---

**김:** 자, 모두 모이셨으니 회의를 시작하겠습니다. 오늘은 신제품 출시 일정과 마케팅 전략에 대해 심도 있게 논의해보도록 하겠습니다. 각 팀에서 준비한 내용들을 공유해주시기 바랍니다.

**이:** 네, 먼저 개발팀의 진행 상황을 상세히 보고드리겠습니다. 현재 주요 기능은 대부분 구현되었고, 내부 테스트를 통해 발견된 버그들을 수정하고 있습니다. 하지만 예상치 못한 문제들이 몇 가지 발생하여 일정이 조금 지연되고 있습니다.

**박:** 어떤 문제들이 발생했는지 구체적으로 말씀해주실 수 있나요?

**이:** 우선 데이터베이스 성능 이슈가 있었습니다. 대량의 데이터를 처리하는 과정에서 응답 속도가 느려지는 문제가 발견되어 최적화 작업을 진행하고 있습니다. 또한, 일부 기능에서 사용자 인터페이스가 복잡하다는 피드백이 있어 이를 개선하고 있습니다.

**최:** 사용자 경험은 매우 중요하니 그 부분은 꼭 개선되어야 할 것 같습니다. 하지만 일정에 큰 영향을 미치는지요?

**이:** 현재로서는 약 일주일 정도 추가 작업이 필요할 것으로 보입니다. 하지만 팀원들이 열심히 노력하고 있으니 최대한 일정을 맞출 수 있도록 하겠습니다.

**김:** 음, 일주일 연기는 마케팅 일정에도 영향을 줄 수 있습니다. 박대리님, 마케팅 측면에서 어떤 영향이 있을까요?

**박:** 네, 출시일이 변경되면 광고 캠페인, 프로모션, 미디어 배포 일정 모두 조정이 필요합니다. 특히 이미 예약된 미디어 인터뷰나 이벤트가 있다면 변경이 어려울 수도 있습니다.

**김:** 그렇다면 출시일을 지키는 것이 최우선이겠군요. 이팀장님, 다른 해결책은 없을까요?

**이:** 개발 인력을 추가로 투입하거나, 일부 비핵심 기능을 후속 업데이트로 미루는 방안을 고려할 수 있습니다. 그러나 새로운 인력을 투입하면 오히려 효율이 떨어질 수 있고, 기능을 미루면 제품 완성도가 낮아질 수 있습니다.

**최:** 혹시 외주를 통해 일부 작업을 맡기는 것은 어떨까요? 전문 업체를 활용하면 시간 단축에 도움이 될 수 있을 것 같습니다.

**이:** 좋은 의견입니다만, 외주 업체와의 협업은 새로운 리스크를 가져올 수 있습니다. 품질 관리나 커뮤니케이션에서 문제가 생길 수 있으니 신중히 검토해야 합니다.

**박:** 마케팅 예산을 일부 조정하여 개발 지원에 투입하는 것은 가능할까요? 예를 들어, 광고 예산을 줄이고 그만큼 개발에 투자하는 식으로요.

**김:** 그 부분은 전략적으로 판단해야 합니다. 제품이 완성되어도 마케팅이 부족하면 시장에서 주목받기 어렵습니다.

**이:** 또한 개발팀의 부담을 줄이기 위해 기존 업무 프로세스를 최적화하거나 자동화 도구를 활용하는 방법도 있습니다. 이를 통해 생산성을 높일 수 있을 것 같습니다.

**최:** 그렇다면 제가 자동화 도구나 생산성 향상 방안에 대해 조사해보겠습니다. 빠른 시일 내에 제안서를 작성하겠습니다.

**김:** 좋습니다. 모두 각자의 위치에서 최선을 다해주시길 바랍니다. 박대리님은 혹시 다른 마케팅 전략이 있을까요?

**박:** 네, 만약 출시일이 연기된다면 그 기간을 활용하여 티저 마케팅이나 고객 참여 이벤트를 진행할 수 있습니다. 이를 통해 소비자들의 관심을 지속적으로 유지할 수 있을 것입니다.

**김:** 좋은 생각입니다. 그 부분은 자세히 계획해주시기 바랍니다.

**이:** 그리고 추가로, 베타 테스트를 진행하여 사용자들의 피드백을 받는 것도 고려하고 있습니다. 이를 통해 제품의 완성도를 높일 수 있을 것입니다.

**박:** 베타 테스트 참가자들에게는 특별한 혜택을 제공하여 참여를 유도하면 효과적일 것 같습니다.

**최:** 베타 테스트 관리와 피드백 수집은 제가 담당하겠습니다. 관련된 플랫폼과 도구를 선정하여 준비하겠습니다.

**김:** 모두 적극적으로 참여해주셔서 감사합니다. 또 다른 논의할 사항이 있나요?

**이:** 한 가지 더 있습니다. 서버 인프라 부분에서 예상 사용자 수를 감안하여 확장이 필요할 것 같습니다. 초기 투자 비용이 증가할 수 있지만, 안정적인 서비스를 제공하기 위해서는 필요한 부분입니다.

**김:** 비용 증가는 어느 정도로 예상되나요?

**이:** 대략 20% 정도의 예산 증액이 필요할 것 같습니다.

**박:** 만약 서버 인프라를 클라우드 서비스를 활용하면 초기 비용을 줄일 수 있지 않을까요?

**이:** 클라우드 서비스를 이용하면 유연성은 높아지지만, 장기적으로는 비용이 더 많이 들 수 있습니다. 또한 보안 이슈도 고려해야 합니다.

**최:** 제가 클라우드 서비스와 자체 서버 운영의 장단점을 비교 분석하여 보고서를 제출하겠습니다.

**김:** 좋습니다. 비용과 효율성을 모두 고려하여 최선의 선택을 해야 합니다.

**박:** 그리고 SNS를 통한 바이럴 마케팅을 강화하기 위해 인플루언서들과의 협업을 추진하고 있습니다. 이에 대한 예산 승인을 부탁드립니다.

**김:** 구체적인 계획서와 예상 효과를 제시해주시면 검토하겠습니다.

**박:** 네, 곧 자료를 준비하겠습니다.

**이:** 마지막으로, 출시 후의 유지보수와 업데이트 계획도 수립하고 있습니다. 이를 위해 추가 인력이 필요할 수 있습니다.

**김:** 인력 채용은 시간이 걸리니 미리 진행하는 것이 좋겠습니다. 인사팀과 협의하여 채용 계획을 세워주세요.

**최:** 혹시 내부 인력의 재배치를 통해 인력 수급을 할 수 있을까요?

**이:** 일부 가능하지만, 전문성이 필요한 부분이 있어 신규 채용이 필요할 것 같습니다.

**김:** 알겠습니다. 인사팀과 협의하여 진행해주시기 바랍니다.

**박:** 그럼 마케팅 팀에서는 출시 후의 홍보 전략도 업데이트하겠습니다. 지속적인 고객 유입을 위해 다양한 채널을 활용할 계획입니다.

**김:** 모두 각자의 역할에서 최선을 다해주시길 바랍니다. 추가로 논의할 사항이 없다면 회의를 마치겠습니다.

**이:** 한 가지 더 있습니다. 사내 교육을 통해 팀원들의 역량을 강화하는 것도 필요할 것 같습니다. 최신 기술이나 트렌드에 대한 교육 프로그램을 제안합니다.

**김:** 좋은 제안입니다. 교육 예산을 확보하여 진행해주시기 바랍니다.

**최:** 제가 교육 프로그램을 조사하고 일정과 내용을 계획하겠습니다.

**박:** 교육 프로그램은 마케팅 팀에도 공유해주시면 감사하겠습니다. 기술적인 이해가 마케팅 활동에도 도움이 될 것입니다.

**이:** 물론입니다. 함께 참여하실 수 있도록 하겠습니다.

**김:** 모두 협력하는 모습이 보기 좋습니다. 그럼 진짜로 오늘 회의를 마치겠습니다.


### 요약:
"""

In [ ]:
BASE_MODEL = "yanolja/EEVE-Korean-Instruct-10.8B-v1.0"

get_summary(BASE_MODEL, prompt)

============= MODEL NAME =============
yanolja/EEVE-Korean-Instruct-10.8B-v1.0


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

============= LOADED MODEL =============
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(40960, 4096)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): Lla

In [ ]:
BASE_MODEL = "nlpai-lab/KULLM3"

get_summary(BASE_MODEL, prompt)

============= MODEL NAME =============
nlpai-lab/KULLM3


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

============= LOADED MODEL =============
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_

In [ ]:
BASE_MODEL = "kimwooglae/WebSquareAI-Instruct-llama-3-8B-v0.5.37"

get_summary(BASE_MODEL, prompt)

============= MODEL NAME =============
kimwooglae/WebSquareAI-Instruct-llama-3-8B-v0.5.37


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


============= LOADED MODEL =============
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): Ll

In [ ]:
BASE_MODEL = "MoaData/Myrrh_solar_10.7b_3.0"

get_summary(BASE_MODEL, prompt)

============= MODEL NAME =============
MoaData/Myrrh_solar_10.7b_3.0


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

============= LOADED MODEL =============
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_